In [1]:
import pandas as pd

In [2]:
pd.set_option("display.float_format", lambda x: '{:,.2f}'.format(x).replace(",", " "))

In [3]:
df = (pd
    .read_html(
        "https://www.resultats-elections.interieur.gouv.fr/presidentielle-2022/FE.html",
        thousands=" ",
        decimal=",",
    )
    [1]
    .assign(
        diff = lambda df_: (df_["Voix"]
            .rolling(window=2)
            .apply(lambda s : -(s.iloc[0] - s.iloc[1]))
        )
    )
)

df

,Liste des candidats,Voix,% Inscrits,% Exprimés,diff
0,M. Emmanuel MACRON,9091680,20.05,27.41,NaN
1,Mme Marine LE PEN,7970921,17.58,24.03,-1 120 759.00
2,M. Jean-Luc MÉLENCHON,7154688,15.78,21.57,-816 233.00
3,M. Éric ZEMMOUR,2313268,5.10,6.97,-4 841 420.00
4,Mme Valérie PÉCRESSE,1576136,3.48,4.75,-737 132.00
5,M. Yannick JADOT,1488536,3.28,4.49,-87 600.00
6,M. Jean LASSALLE,1076892,2.38,3.25,-411 644.00
7,M. Fabien ROUSSEL,773754,1.71,2.33,-303 138.00
8,M. Nicolas DUPONT-AIGNAN,702708,1.55,2.12,-71 046.00
9,Mme Anne HIDALGO,576058,1.27,1.74,-126 650.00


In [4]:
df.dtypes

Liste des candidats     object
Voix                     int64
% Inscrits             float64
% Exprimés             float64
diff                   float64
dtype: object

In [5]:
url_2017 = "https://www.interieur.gouv.fr/Elections/Les-resultats/Presidentielles/elecresult__presidentielle-2017/(path)/presidentielle-2017//FE.html"

df_2017 = (pd
    .read_html(url_2017, encoding="utf-8",
        thousands=" ",
        decimal=",",)
    [2]
)

df_2017

,Liste des candidats,Voix,% Inscrits,% Exprimés
0,M. Emmanuel MACRON,8656346,18.19,24.01
1,Mme Marine LE PEN,7678491,16.14,21.30
2,M. François FILLON,7212995,15.16,20.01
3,M. Jean-Luc MÉLENCHON,7059951,14.84,19.58
4,M. Benoît HAMON,2291288,4.82,6.36
5,M. Nicolas DUPONT-AIGNAN,1695000,3.56,4.70
6,M. Jean LASSALLE,435301,0.91,1.21
7,M. Philippe POUTOU,394505,0.83,1.09
8,M. François ASSELINEAU,332547,0.70,0.92
9,Mme Nathalie ARTHAUD,232384,0.49,0.64


In [6]:
(pd
    .merge(df, df_2017, on="Liste des candidats", suffixes=('_2022', '_2017'))
    [["Liste des candidats", "Voix_2017", "Voix_2022"]]
    .assign(
        difference = lambda df_: df_["Voix_2022"] - df_["Voix_2017"],
        pct = lambda df_: df_["Voix_2022"] / df_["Voix_2017"] - 1
    )
    .set_index("Liste des candidats")
    .style
        .format(thousands=" ",
            formatter={'pct': '{:,.2%}'.format}
#            formatter = {('difference'): lambda x: "🔼 "+str(x) if x > 0 else "🔽 "+str(x) }
        )
)

,Voix_2017,Voix_2022,difference,pct
Liste des candidats,,,,
M. Emmanuel MACRON,8 656 346,9 091 680,435 334,5.03%
Mme Marine LE PEN,7 678 491,7 970 921,292 430,3.81%
M. Jean-Luc MÉLENCHON,7 059 951,7 154 688,94 737,1.34%
M. Jean LASSALLE,435 301,1 076 892,641 591,147.39%
M. Nicolas DUPONT-AIGNAN,1 695 000,702 708,-992 292,-58.54%
M. Philippe POUTOU,394 505,257 659,-136 846,-34.69%
Mme Nathalie ARTHAUD,232 384,191 618,-40 766,-17.54%
